In [21]:
import duckdb

In [ ]:
# Let's create a relational database with DuckDB from NYC yellow taxi data 
# from January 2023.
con = duckdb.connect("nyc_taxi.duckdb")

con.execute("""
    CREATE OR REPLACE TABLE trips AS 
    SELECT * FROM read_parquet('yellow_tripdata_2023-01.parquet')
            """)

# Next, we need to add one more table into the dataset for location IDs. 
con.execute("""
    CREATE OR REPLACE TABLE locations AS
    SELECT * FROM read_csv_auto('taxi_zone_lookup.csv')
            """)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [23]:
# Let's make sure our tables are in the database.
print(con.execute("""
    SHOW TABLES                  
""").fetchdf())


        name
0  locations
1      trips


In [24]:
# Now, lets look at the contents of the locations table.
print(con.execute("""
    SELECT * FROM locations                  
""").fetchdf())

     LocationID        Borough                     Zone service_zone
0             1            EWR           Newark Airport          EWR
1             2         Queens              Jamaica Bay    Boro Zone
2             3          Bronx  Allerton/Pelham Gardens    Boro Zone
3             4      Manhattan            Alphabet City  Yellow Zone
4             5  Staten Island            Arden Heights    Boro Zone
..          ...            ...                      ...          ...
260         261      Manhattan       World Trade Center  Yellow Zone
261         262      Manhattan           Yorkville East  Yellow Zone
262         263      Manhattan           Yorkville West  Yellow Zone
263         264        Unknown                      N/A          N/A
264         265            N/A           Outside of NYC          N/A

[265 rows x 4 columns]


In [25]:
# Now look at the contents of the trips table. This one is massive, so only print the first 3.
print(con.execute("""
    SELECT * FROM trips LIMIT 3                 
""").fetchdf())

   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2023-01-01 00:32:10   2023-01-01 00:40:36              1.0   
1         2  2023-01-01 00:55:08   2023-01-01 01:01:27              1.0   
2         2  2023-01-01 00:25:04   2023-01-01 00:37:49              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           0.97         1.0                  N           161           141   
1           1.10         1.0                  N            43           237   
2           2.51         1.0                  N            48           238   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \
0             2          9.3    1.0      0.5         0.0           0.0   
1             1          7.9    1.0      0.5         4.0           0.0   
2             1         14.9    1.0      0.5        15.0           0.0   

   improvement_surcharge  total_amount  congestion_surcharge  airport_fee  
0        

In [26]:
# Determine total number of trips in January 2023, the average fare price,
# and tip amount as an absolute value and as a percentage.

df = con.execute("""
    SELECT
                  COUNT(*) AS total_trips,
                  AVG(fare_amount) AS avg_fare,
                  AVG(tip_amount) AS avg_tip,
                  (AVG(tip_amount)/AVG(fare_amount)) * 100 AS tip_perc

    FROM trips;             
""").fetchdf()

df = df.round(2)
print(df)

# Let's also save the total trips for this month. We will need this in a minute.
total_trips = df.iloc[0,0]



   total_trips  avg_fare  avg_tip  tip_perc
0      3066766     18.37     3.37     18.34


In [27]:
# How many trips in January 2023 featured Manhattan as either a pickup or dropoff location?
with open("man_at_all.sql") as f:
    query = f.read()
df =con.execute(query).fetchdf()

man_trips = len(df)

print(f"Percentage of cab rides where they either entered or left Manhattan: {round((man_trips/total_trips)*100,1)}%")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Percentage of cab rides where they either entered or left Manhattan: 94.5%


In [28]:
# How many were outside Manhattan?
with open("outside_man.sql") as f:
    query = f.read()
df =con.execute(query).fetchdf()

outside_man_trips = len(df)
print(f"Percentage of cab rides outside Manhattan: {round((outside_man_trips/total_trips)*100,1)}%")

Percentage of cab rides outside Manhattan: 5.5%


In [29]:
# How many were inside Manhattan?
with open("inside_man.sql") as f:
    query = f.read()
df =con.execute(query).fetchdf()

inside_man_trips = len(df)
print(f"Percentage of cab rides inside Manhattan: {round((inside_man_trips/total_trips)*100,1)}%")

Percentage of cab rides inside Manhattan: 82.9%
